In [59]:
import pandas as pd
import numpy as np

io = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(io, match='.+', flavor='bs4', header=None, index_col=None, skiprows=None, attrs=None, parse_dates=False, tupleize_cols=None, thousands=', ', encoding=None, decimal='.', converters=None, na_values=None, keep_default_na=True)
df = data[0]

#replace top numbers with headers
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0))
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [49]:
df.dropna(subset=["Borough"], axis=0, inplace=True)
df_postcodes = df['Postcode']
df_postcodes.head()

#if a cell has a borough but no neighbourhood, the borough will also be the neighbourhood

# Create a list of indices of the cells where the value of Column2 is "Not assigned"
idx_to_change = df.loc[df['Neighbourhood'] == "Not assigned"].index
idx_to_change

# Iterate the list of indicies and set Neighbourhood name to the Column name at the given index
for i in idx_to_change:
    #print(df['Neighbourhood'][i])
    df['Neighbourhood'][i] = df['Borough'][i]
df

#drop rows that have both the borough and the neighbourhood not assigned
#change Not assigned to NaN
df.replace("Not assigned", np.nan, inplace = True)
#drop all NaN rows
df.dropna()

# reset index, because we droped rows
df.reset_index(drop=True, inplace=True)

df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [56]:
#Combine two rows with same code into one row with the neighborhoods separated with a comma
Grouped = df.groupby(['Postcode', 'Borough'], as_index = False, sort = False).agg(', '.join)
Grouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [58]:
#view number of rows and columns in dataframe
Grouped.shape

(103, 3)

In [64]:
#get latitude and longitude of postal codes from csv file
filename = "http://cocl.us/Geospatial_data"
df = pd.read_csv(filename)
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
#merge the lat and longitude data with the original pandas df
df = pd.concat([df, dummy_variable_1], axis=1)